# Classifying Traffic Signs with OpenCV

For this project we will be trying split the traffic sign dataset into 3 separate classes:
- Stop signs
- Red Circles
- Blue rectangles/squares 

### Imports

In [1]:
import cv2 as cv
import numpy as np
import os, os.path
import shutil
from ImageList import ImageList
from Image import Image

In [324]:
# Results directories reset
shutil.rmtree('results/blue_squares')
shutil.rmtree('results/red_circles')
shutil.rmtree('results/stop_signs')
shutil.rmtree('results/unknowns')
os.mkdir('results/blue_squares')
os.mkdir('results/red_circles')
os.mkdir('results/stop_signs')
os.mkdir('results/unknowns')

### Useful functions

This function reads the images from the dataset and transfers them to a python list removing possible duplicates.

In [325]:
def get_image_list():
    # Saves images into a list containing objects of class Image which contains traffic sign info 
    image_list = ImageList() 
    for root, _, files in os.walk('./dataset'):
        for file in files:
            split_file_path = os.path.join(root, file).split(os.sep)
            img_name = split_file_path[-1]
            _type = split_file_path[-2][:-1]
            # Prevent image duplicates
            idx = image_list.contains_img(img_name)
            if idx >= 0:
                image = image_list.get_image(idx)
                image.add_type(_type)
            else:
                image = Image(img_name, _type)
                image_list.add_image(image)
    
    return image_list

The following function takes a gray level picture (segmented_img) and applies several transformations to improve the chances of a sign being detected.

Operations performed:
 - Smoothing
 - Morphological closing (dilation followed by erosion) with a 5x5 kernel (to blue signs)
 - Canny edge detector (to blue signs)

After this contours are searched that may match a blue square or a stop sign. If a contour with more than 8 edges is found then we verify if it is a circle.

This returns the final picture with the contours and the predictions.

In [326]:
def process_image(segmented_img, original_img, color):
    segmented_img = smooth(segmented_img)
    
    if color == "blue":
        kernel = np.ones((5,5), np.uint8)
        segmented_img = cv.dilate(segmented_img, kernel, iterations=3)
        segmented_img = cv.erode(segmented_img, kernel, iterations=3)
        segmented_img = cv.Canny(segmented_img, 150, 200) 

    final, prediction = search_contours(segmented_img, original_img.copy(), color)
    
    if type(prediction) != str: # Prediction is returned as a number if found contour has more than 8 edges
        final, prediction = find_circles(segmented_img, original_img.copy(), int(prediction))
        
    return final, prediction

For image smoothing we use a bilateral filter

In [327]:
# Remove image noise with bilateral filter (better at preserving edges)
def smooth(img):
    return cv.bilateralFilter(img, 5, 75, 75)

The following function increases the brightness of the provided image so that colors can be better identified. Isn't used in pipeline because of worse results

In [328]:
# Increases image brightness
def increase_brightness(img, value=30):
    hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    h, s, v = cv.split(hsv)

    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value

    final_hsv = cv.merge((h, s, v))
    img = cv.cvtColor(final_hsv, cv.COLOR_HSV2BGR)
    return img

The segment function looks for the color specified in the argument and creates a black and white mask with the pixels of the desired color set to white.

Red is found at both ends of the HSV hue spectrum so two masks are needed for the red color.

In [329]:
#  Segments reds and blues of an image
def segment(img, color):
    img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    if color == 'red':
        # lower mask (0-10)
        lower_red = np.array([0,90,50])
        upper_red = np.array([10,255,255])
        mask0 = cv.inRange(img_hsv, lower_red, upper_red)  

        # upper mask (160-180)
        lower_red = np.array([160,90,50])
        upper_red = np.array([180,255,255])
        mask1 = cv.inRange(img_hsv, lower_red, upper_red)

        mask = mask0 | mask1
    elif color == 'blue':
        lower_blue = np.array([90,140,50])
        upper_blue = np.array([130,255,255])
        mask = cv.inRange(img_hsv, lower_blue, upper_blue) 

    segmented_img = cv.bitwise_and(img_hsv, img_hsv, mask = mask)
    segmented_img = cv.cvtColor(segmented_img, cv.COLOR_BGR2GRAY)
    return segmented_img    


This function finds the largest contour in a black and white image. It detects and draws the contours if they match the specified criteria, minimum area, minimum perimeter and maximum perimeter.

In [330]:
# Find the contour with the largest area in a gray scale image
def find_largest_contour(segmented_img, original_img):
    contours, _ = cv.findContours(segmented_img,2,1)
    big_contour = []
    max = 0
    for i in contours:
        area = cv.contourArea(i)
        if(area > max):
            max = area
            big_contour = i 
    if max > 1000 and len(big_contour) > 10 and len(big_contour) < 1200:
        final = cv.drawContours(original_img, big_contour, -1, (0,255,0), 3)
    else:
        final = original_img

    return final, big_contour

The following function verifies if the segmented image has any white pixels, calls the find_largest_contour function and processes the return.

It counts the number of edges found on the segmented image classifies the sign according to it.

In [331]:
# Find a blue square
def search_contours(segmented_img, original_img, color):
    if cv.countNonZero(segmented_img) == 0:
        return original_img, 'unrecognised'
    final, contour = find_largest_contour(segmented_img, original_img)
    if len(contour) > 100:
        peri = cv.arcLength(contour, True)
        approx = cv.approxPolyDP(contour, 0.01 * peri, True)
        if len(approx) > 8: # circle possibly
            return final, cv.contourArea(contour)
        return final, classify_sign(approx, color)

    return final, 'unrecognised'

Classifies the sign according to the number of edges present

In [332]:
def classify_sign(approx_cnt, color):
    if color == 'blue':
        if len(approx_cnt) == 4:
            return 'blue_square'
    elif color == 'red':
        if len(approx_cnt) == 8:
            return 'stop_sign'
    return 'unrecognised'

The following function applies further processing and attempts to find circles in the picture.

In [333]:
def find_circles(img, original_img, radius):
    # Reduce white noise and enhance shapes
    kernel = np.ones((3, 3), np.uint8)
    img = cv.dilate(img, kernel, iterations = 3)
    img = cv.erode(img, kernel, iterations = 3)
    #Tolerance
    tol = 1
    circles = cv.HoughCircles(img, cv.HOUGH_GRADIENT, 1, radius * 2 - tol, param1=100, param2=45, minRadius=5, maxRadius=radius + tol)

    if circles is not None:
        circles = np.uint16(np.around(circles))

        for i in circles[0, :]:
            center = (i[0], i[1])
            r = i[2]
            cv.circle(original_img, center, r, (0, 255, 0), 2)
        return original_img, 'red_circle'
    else:
        return original_img, 'unrecognised'


#### Paths and image list

In [334]:
dataset_path = 'dataset'
results_path = 'results'

# Image List
img_list = get_image_list()

### Smoothing applied in different solutions

In order to reduce the amount of unrecognised traffic signs in the dataset images we tried to increase images smoothing until the results started getting worse. Thats why the "amount" of smoothing done in original images for blue segmented image processing is higher than the "amount" used in original images for red segmented images. 

Different amount of bilateral (filter) smoothing was applied in original images before segmenting 

In [335]:
# Counters
cnt_un1 = 0    # Counter for the amount of images unrecognised prior to using more smoothing if it is unrecognised
cnt_un2 = 0      # Counter for the amount of images unrecognised after using more smoothing

# Image Lists 
img_list1 = img_list.copy() # Image list for low amount of smoothing
img_list2 =  img_list.copy() # Image list for higher amount of smoothing

In [336]:
inp = 0
while inp <= 0 or inp > 3:
    inp = int(input('Select solution to be written to the results directory (1- Normal Smoothing; 2- High Smoothing): '))

#### Normal amount

Smoothing only applied once in original image and then on the segmented one inside process_image function

In [337]:
for i in range(0, img_list1.len()):
    # Get img object from img_list
    img_obj = img_list1.get_image(i)
    
    # Get path of that image -> {type(qualquer um serve)}/{filename}
    img_path = img_obj.types[0] + 's' + os.sep + img_obj.filename

    # Image in the dataset path
    img_path_in_dataset = dataset_path + os.sep + img_path

    # Read img
    img = cv.imread(img_path_in_dataset)

    # Keep an original copy
    original = img.copy()

    # Smooth image
    img = smooth(img)

    # Segment it by filtering the blue and red color for blue and red signs
    img_blue, img_red = segment(img, 'blue'), segment(img, 'red')

    # Variable to check if any sign is already detected for it not to be classified as unrecognised in that case
    sign_detected = False

    # Process blue segmented img
    final, classification = process_image(img_blue.copy(), img.copy(), 'blue')

    if classification == 'blue_square':
        img_obj.add_classification(classification)
        if inp == 1:
            cv.imwrite(os.path.join(results_path, classification +
                                    's', img_obj.filename), final)
        sign_detected = True

    final, classification = process_image(img_red.copy(), img.copy(), 'red')

    if classification == 'stop_sign' or classification == 'red_circle':
        img_obj.add_classification(classification)
        if inp == 1:
            cv.imwrite(os.path.join(
                results_path, classification + 's', img_obj.filename), final)
        sign_detected = True

    if sign_detected == False:
        cnt_un1 += 1
        img_obj.add_classification(classification)
        if inp == 1:
            cv.imwrite(os.path.join(
                results_path, 'unknowns', img_obj.filename), final)

#### Higher amount

Smoothing applied more times in original image if normal smoothing didn't detect any traffic signs

In [338]:
for i in range(0, img_list2.len()):
    # Get img object from img_list
    img_obj = img_list2.get_image(i)

    # Get path of that image -> {type(qualquer um serve)}/{filename}
    img_path = img_obj.types[0] + 's' + os.sep + img_obj.filename

    # Image in the dataset path
    img_path_in_dataset = dataset_path + os.sep + img_path

    # Read img
    img = cv.imread(img_path_in_dataset)

    # Keep an original copy
    original = img.copy()

    # Smooth image
    img = smooth(img)

    # Segment it by filtering the blue and red color for blue and red signs
    img_blue, img_red = segment(img, 'blue'), segment(img, 'red')

    # Variable to check if any sign is already detected for it not to be classified as unrecognised in that case
    sign_detected = False

    # Process blue segmented img
    final, classification = process_image(img_blue.copy(), img.copy(), 'blue')

    # Process first iteration result
    if classification == 'blue_square':
        img_obj.add_classification(classification)
        if inp == 2:
            cv.imwrite(os.path.join(results_path, classification +
                                    's', img_obj.filename), final)
        sign_detected = True
    elif classification == 'unrecognised':  # if no sign detected do more smoothing
        temp = smooth(smooth(smooth(img_blue)))
        final, classification = process_image(temp, img.copy(), 'blue')
        if classification == 'blue_square':
            img_obj.add_classification(classification)
            if inp == 2:
                cv.imwrite(os.path.join(results_path, classification +
                                        's', img_obj.filename), final)
            sign_detected = True

    # Process red segmented img
    final, classification = process_image(img_red.copy(), img.copy(), 'red')

    # Process first iteration result
    if classification == 'stop_sign' or classification == 'red_circle':
        img_obj.add_classification(classification)
        if inp == 2:
            cv.imwrite(os.path.join(
                results_path, classification + 's', img_obj.filename), final)
        sign_detected = True
    elif classification == 'unrecognised':  # if no sign detected do more smoothing
        temp = smooth(img_red)
        final, classification = process_image(temp, img.copy(), 'red')
        if classification == 'stop_sign' or classification == 'red_circle':
            img_obj.add_classification(classification)
            if inp == 2:
                cv.imwrite(os.path.join(results_path, classification +
                                        's', img_obj.filename), final)
            sign_detected = True

    if sign_detected == False:
        cnt_un2 += 1
        img_obj.add_classification(classification)
        if inp == 2:
            cv.imwrite(os.path.join(
                results_path, 'unknowns', img_obj.filename), final)


Amount of unknowns (no traffic signs detected) and accuracy for each traffic sign type. This accuracy is calculated in a method of the ImageList class by dividing the amount of correct classifications by the amount expected.

In [339]:
print('==========No Traffic Signs Detected==========')
print('Normal amount of smoothing: ' + str(cnt_un1))
print('Higher amount of smoothing: ' + str(cnt_un2))
print('==========Detection Accuracy for Red Circles==========')
print('Normal amount of smoothing: ' + str(img_list1.evaluate_classification_accuracy('red_circle')))
print('Higher amount of smoothing: ' + str(img_list2.evaluate_classification_accuracy('red_circle')))
print('==========Detection Accuracy for Blue Squares==========')
print('Normal amount of smoothing: ' + str(img_list1.evaluate_classification_accuracy('blue_square')))
print('Higher amount of smoothing: ' + str(img_list2.evaluate_classification_accuracy('blue_square')))
print('==========Detection Accuracy for Stop Signs==========')
print('Normal amount of smoothing: ' + str(img_list1.evaluate_classification_accuracy('stop_sign')))
print('Higher amount of smoothing: ' + str(img_list2.evaluate_classification_accuracy('stop_sign')))

==========No Traffic Signs Detected==========
Normal amount of smoothing: 62
Higher amount of smoothing: 49
==========Detection Accuracy for Red Circles==========
Normal amount of smoothing: 40.0
Higher amount of smoothing: 52.0
==========Detection Accuracy for Blue Squares==========
Normal amount of smoothing: 47.05882352941177
Higher amount of smoothing: 52.94117647058823
==========Detection Accuracy for Stop Signs==========
Normal amount of smoothing: 66.66666666666667
Higher amount of smoothing: 68.62745098039215


#### Pipeline Example (using a image with a red circle sign)

In [ ]:
img_path= os.path.join()
img =cv.imread(img_path)
cv.imshow("original",img)

img=smooth(img)
cv.imshow("first smooth result",img)

img_red=segment(img,'red')
cv.imshow("red filter applied",img_red)

final, classification = process_image(img_red.copy(), img.copy(), 'red')
if classification == 'stop_sign' or classification == 'red_circle':
    cv.imshow("final classified image(red circle)",final)
elif classification=='unrecognised':
    cv.imshow("second smooth result",final)
    temp=smooth(img_red)
    final, classification = process_image(temp, img.copy(), 'red')
    cv.imshow("final classified image after further smoothing(red circle)",final)


cv.waitKey(0)
cv.destroyAllWindows()